## Load data

In [43]:
import os
import json
import pickle
import numpy as np
import random, math, re, pandas as pd
from typing import List
from collections import Counter, OrderedDict
from itertools import chain

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from itertools import product

In [44]:
import random

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

g = torch.Generator()
g.manual_seed(SEED)

In [45]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Using {device} as device.")

Using cpu as device.


In [46]:
with open("./gutenberg.txt", encoding="utf-8") as f:
    sentences_raw = f.readlines()

## Preprocessing Text

In [47]:
def preprocess_text(text: str):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.split() if text else []


In [48]:
sentences = []
for line in sentences_raw:
    tokens = preprocess_text(line)
    if tokens:
        sentences.append(tokens)
print(len(sentences), "sentences")

5298 sentences


### Subsampling + Vocab

- Applied subsampling to the corpus to reduce the influence of very frequent words. For each word, it computes a keep probability based on its frequency, then randomly drops words according to that probability to create subsampled_sentences -> reduce influence of words like (the, is, was, of, and ...)

In [49]:
from collections import Counter

counter = Counter()
total_tokens = 0
for sent in sentences:
    counter.update(sent)
    total_tokens += len(sent)

freq = {w: c / total_tokens for w, c in counter.items()}
word_counts = OrderedDict(
    sorted(counter.items(), key=lambda x: x[1], reverse=True)
)
print("Distinct tokens:", len(word_counts))


Distinct tokens: 5014


In [50]:
t = 1e-3

def keep_prob(word):
    f = freq[word]
    return math.sqrt(t / f) + (t / f)

subsampled_sentences = []
for sent in sentences:
    new_sent = []
    for w in sent:
        if w not in freq:
            new_sent.append(w)
            continue
        if random.random() < keep_prob(w):
            new_sent.append(w)
    if new_sent:
        subsampled_sentences.append(new_sent)

print("Before subsampling:", sum(len(s) for s in sentences))
print("After subsampling:", sum(len(s) for s in subsampled_sentences))


Before subsampling: 61920
After subsampling: 43565


In [51]:
# Check which words are removed

counter_before = Counter()
for sent in sentences:
    counter_before.update(sent)

counter_after = Counter()
for sent in subsampled_sentences:
    counter_after.update(sent)

In [52]:
removed_stats = []

for w, c_before in counter_before.items():
    c_after = counter_after.get(w, 0)
    if c_before == 0:
        continue
    drop = c_before - c_after
    drop_ratio = drop / c_before  

    removed_stats.append((w, c_before, c_after, drop, drop_ratio))

removed_stats.sort(key=lambda x: x[4], reverse=True)

# top 30
for w, cb, ca, drop, ratio in removed_stats[:30]:
    print(f"{w:15s} before={cb:5d}, after={ca:5d}, dropped={drop:5d}, ratio={ratio:6.2%}")


the             before= 3633, after=  530, dropped= 3103, ratio=85.41%
of              before= 2629, after=  481, dropped= 2148, ratio=81.70%
to              before= 2197, after=  443, dropped= 1754, ratio=79.84%
and             before= 2164, after=  468, dropped= 1696, ratio=78.37%
that            before=  978, after=  288, dropped=  690, ratio=70.55%
a               before= 1014, after=  305, dropped=  709, ratio=69.92%
in              before= 1055, after=  320, dropped=  735, ratio=69.67%
it              before=  893, after=  308, dropped=  585, ratio=65.51%
is              before=  843, after=  304, dropped=  539, ratio=63.94%
be              before=  714, after=  266, dropped=  448, ratio=62.75%
or              before=  626, after=  250, dropped=  376, ratio=60.06%
as              before=  634, after=  257, dropped=  377, ratio=59.46%
by              before=  570, after=  232, dropped=  338, ratio=59.30%
his             before=  618, after=  256, dropped=  362, ratio=58.58%
for   

In [53]:
class Vocab:
    def __init__(
        self,
        word_counts: OrderedDict, # vocabular is based on word counts
        min_freq: int = 1, # min times a word must appear in corpus (rare words might not be worth considering)
        max_size: int = None, # we can limit the amount of words as well 
        specials: List[str] = None, # any other special tokens we may want to add, like padding tokens
        unk_token: str = "<unk>" # reserved token for when we run into words not in the vocabulary
    ):
        self.word_counts = word_counts
        self.min_freq = min_freq
        self.max_size = max_size
        self.unk_token = unk_token
        self.specials = list(specials) if specials else []

        if self.unk_token not in self.specials:
            self.specials.insert(0, self.unk_token) # unknown token should always be included

        self.token2idx = {}
        self.idx2token = []

        self._prepare_vocab()


    def __len__(self):
        return len(self.idx2token)
    

    def __contains__(self, value):
        return value in self.idx2token


    def _prepare_vocab(self):
        """Processes input OrderedDict: Filters based on min_freq & adds special tokens."""
        vocab_list = self.specials.copy()  # Copy specials to avoid modifying original list

        # filter words based on min_freq and add to vocab
        filtered_words = [
            word
            for word, freq in self.word_counts.items()
            if freq >= self.min_freq and word not in self.specials
        ]

        # enforcing max vocab size constraint
        if self.max_size is not None:
            n_to_keep = self.max_size - len(self.specials) # special tokens take up spaces
            filtered_words = filtered_words[:n_to_keep]

        # creating final vocab list
        vocab_list.extend(word for word in filtered_words)

        # create look up tables
        self.idx2token = vocab_list
        self.token2idx = {word: idx for idx, word in enumerate(vocab_list)}


    def get_token(self, idx: int) -> str:
        """Returns the token corresponding to an index. Raises error if index is out of range."""
        if 0 <= idx < len(self.idx2token):
            return self.idx2token[idx]
        raise IndexError(f"Index {idx} is out of range for vocabulary size {len(self.idx2token)}")


    def get_index(self, token: str) -> int:
        """Returns the index corresponding to a token. Defaults to unk_token if missing."""
        return self.token2idx.get(token, self.token2idx[self.unk_token])  # return unk_token index if word is not in vocab


    def get_tokens(self, indices: List[int]) -> List[str]:
        """Converts a list of indices into a list of tokens."""
        return [self.get_token(idx) for idx in indices]


    def get_indices(self, tokens: List[str]) -> List[int]:
        """Converts a list of tokens into a list of indices."""
        return [self.get_index(token) for token in tokens]

In [54]:
def pad_sentences(sentences: List[List[str]], context_length: int, pad_token: str = "<pad>") -> List[List[str]]:
    """
    Pads each sentence to fit the context window length with the literal string "<pad>".
    
    Args:
        sentences: A list of sentences, where each sentence is a list of tokens.
        context_length: The number of tokens to either side of the target token.

    Returns:
        A list of padded sentences.
    """
    padded_sentences = []
    for sentence in sentences:
        padded_sentence = [pad_token] * context_length + sentence + [pad_token] * context_length
        padded_sentences.append(padded_sentence)
    
    return padded_sentences

In [55]:
counter_sub = Counter()
for sent in subsampled_sentences:
    counter_sub.update(sent)

word_counts_sub = OrderedDict(
    sorted(counter_sub.items(), key=lambda x: x[1], reverse=True)
)

MIN_FREQ = 5
vocab = Vocab(
    word_counts=word_counts_sub,
    min_freq=MIN_FREQ,
    max_size=None,
    specials=["<pad>"],
    unk_token="<unk>",
)

In [56]:
# creating a vocabulary
print(f"Size of Vocabulary: {len(vocab):,}")

Size of Vocabulary: 1,183


In [57]:
for idx in [0, 1, 5, 100]:
    print(f"Index {idx} corresponds to `{vocab.get_token(idx)}`")

Index 0 corresponds to `<unk>`
Index 1 corresponds to `<pad>`
Index 5 corresponds to `to`
Index 100 corresponds to `up`


In [58]:
# Exclude <pad> and <unk> from both target and context.
PAD_IDX = vocab.get_index("<pad>") 
UNK_IDX = vocab.get_index("<unk>")

def generate_skipgram(sentences, context_length, vocab):
    targets = []
    contexts = []

    for sentence in sentences:
        enc_sentence = vocab.get_indices(sentence)

        for target_idx in range(context_length, len(enc_sentence) - context_length):
            target = enc_sentence[target_idx]
            if target in (PAD_IDX, UNK_IDX):
                continue

            for offset in range(-context_length, context_length + 1):
                if offset == 0:
                    continue
                context_idx = target_idx + offset
                context = enc_sentence[context_idx]
                if context in (PAD_IDX, UNK_IDX):
                    continue

                targets.append(target)
                contexts.append(context)

    return torch.tensor(targets), torch.tensor(contexts)


## Dataset / DataLoader

In [59]:
from torch.utils.data import Dataset, DataLoader

class SkipGramDataset(Dataset):
    def __init__(self, targets, contexts):
        self.targets = targets
        self.contexts = contexts

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.targets[idx], self.contexts[idx]


## Define Model

In [60]:
import torch
import torch.nn as nn

class SkipGramModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, embedding_dim)
        self.out = nn.Linear(embedding_dim, vocab_size)

    def forward(self, target_indices):
        # target_indices: [batch_size]
        emb = self.emb(target_indices)         # [batch_size, embedding_dim]
        logits = self.out(emb)                # [batch_size, vocab_size]
        return logits


## Model Train / Evaluation

In [61]:
def most_similar_from_embeddings(word, embeddings, vocab, top_k=10):
    if word not in vocab.token2idx:
        return []  

    idx = vocab.get_index(word)
    vec = embeddings[idx]  # [embedding_dim]

    sims = F.cosine_similarity(vec.unsqueeze(0), embeddings)  # [vocab_size]
    topk = torch.topk(sims, top_k + 1) 

    results = []
    for score, i in zip(topk.values.tolist(), topk.indices.tolist()):
        token = vocab.get_token(i)
        if token == word:
            continue
        results.append((token, score))
        if len(results) >= top_k:
            break
    return results  # [(neighbor, score), ...]


In [62]:
from collections import Counter, OrderedDict

counter = Counter()
for sent in sentences:         
    counter.update(sent)

word_counts = OrderedDict(
    sorted(counter.items(), key=lambda x: x[1], reverse=True)
)

print("Number of distinct tokens:", len(word_counts))
list(word_counts.items())[:10]  


Number of distinct tokens: 5014


[('the', 3633),
 ('of', 2629),
 ('to', 2197),
 ('and', 2164),
 ('in', 1055),
 ('a', 1014),
 ('that', 978),
 ('it', 893),
 ('is', 843),
 ('be', 714)]

In [63]:
window_list       = [2, 4]
embedding_list    = [50, 100]
min_freq_list     = [5]          
batch_size_list   = [64]
n_epochs_list     = [2]

# Words to check
probe_words = ["freedom", "law", "power"]
TOP_K = 5 


In [64]:
results = [] 

config_id = 0

for CONTEXT_WINDOW, EMBEDDING_SIZE, MIN_FREQ, BATCH_SIZE, N_EPOCHS in product(
    window_list, embedding_list, min_freq_list, batch_size_list, n_epochs_list
):
    config_id += 1
    print(f"\n=== Config {config_id} ===")
    print(f"WINDOW={CONTEXT_WINDOW}, EMB_DIM={EMBEDDING_SIZE}, MIN_FREQ={MIN_FREQ}, "
          f"BATCH={BATCH_SIZE}, EPOCHS={N_EPOCHS}")
    
    # skip-gram pair
    targets, contexts = generate_skipgram(sentences=sentences, context_length=CONTEXT_WINDOW,vocab=vocab)

    dataset = SkipGramDataset(targets, contexts)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, generator=g)
    
    # model define
    model = SkipGramModel(
        vocab_size=len(vocab),
        embedding_dim=EMBEDDING_SIZE,
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    # model train
    model.train()
    for epoch in range(N_EPOCHS):
        total_loss = 0.0
        for batch_targets, batch_contexts in dataloader:
            batch_targets = batch_targets.to(device)
            batch_contexts = batch_contexts.to(device)

            logits = model(batch_targets)
            loss = criterion(logits, batch_contexts)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * batch_targets.size(0)

        avg_loss = total_loss / len(dataset)
        print(f"  Epoch {epoch+1}/{N_EPOCHS} - Loss: {avg_loss:.4f}")

    # most_similar results based on trained embeddings
    embeddings = model.emb.weight.detach().cpu()

    for word in probe_words:
        neighbors = most_similar_from_embeddings(word, embeddings, vocab, top_k=TOP_K)

        if not neighbors:
            results.append({
                "config_id": config_id,
                "window": CONTEXT_WINDOW,
                "embedding_dim": EMBEDDING_SIZE,
                "min_freq": MIN_FREQ,
                "batch_size": BATCH_SIZE,
                "epochs": N_EPOCHS,
                "word": word,
                "neighbor_rank": None,
                "neighbor": None,
                "similarity": None,
            })
            continue

        for rank, (nbr, score) in enumerate(neighbors, start=1):
            results.append({
                "config_id": config_id,
                "window": CONTEXT_WINDOW,
                "embedding_dim": EMBEDDING_SIZE,
                "min_freq": MIN_FREQ,
                "batch_size": BATCH_SIZE,
                "epochs": N_EPOCHS,
                "word": word,
                "neighbor_rank": rank,
                "neighbor": nbr,
                "similarity": score,
            })

df_results = pd.DataFrame(results)



=== Config 1 ===
WINDOW=2, EMB_DIM=50, MIN_FREQ=5, BATCH=64, EPOCHS=2


  Epoch 1/2 - Loss: 5.9218
  Epoch 2/2 - Loss: 5.2842

=== Config 2 ===
WINDOW=2, EMB_DIM=100, MIN_FREQ=5, BATCH=64, EPOCHS=2
  Epoch 1/2 - Loss: 5.7862
  Epoch 2/2 - Loss: 5.1794

=== Config 3 ===
WINDOW=4, EMB_DIM=50, MIN_FREQ=5, BATCH=64, EPOCHS=2
  Epoch 1/2 - Loss: 5.9686
  Epoch 2/2 - Loss: 5.4223

=== Config 4 ===
WINDOW=4, EMB_DIM=100, MIN_FREQ=5, BATCH=64, EPOCHS=2
  Epoch 1/2 - Loss: 5.8641
  Epoch 2/2 - Loss: 5.3553


In [65]:
# window_size = 2, embedding_dim = [50, 100]
df_results[df_results['window'] == 2]

,config_id,window,embedding_dim,min_freq,batch_size,epochs,word,neighbor_rank,neighbor,similarity
0,1,2,50,5,64,2,freedom,1,your,0.472956
1,1,2,50,5,64,2,freedom,2,governing,0.472150
2,1,2,50,5,64,2,freedom,3,cease,0.452665
3,1,2,50,5,64,2,freedom,4,together,0.413366
4,1,2,50,5,64,2,freedom,5,assembling,0.410174
5,1,2,50,5,64,2,law,1,damages,0.499466
6,1,2,50,5,64,2,law,2,laws,0.493520
7,1,2,50,5,64,2,law,3,then,0.473101
8,1,2,50,5,64,2,law,4,atque,0.441182
9,1,2,50,5,64,2,law,5,remain,0.440887


In [66]:
# window_size = 4, embedding_dim = [50, 100]
df_results[df_results['window'] == 4]

,config_id,window,embedding_dim,min_freq,batch_size,epochs,word,neighbor_rank,neighbor,similarity
30,3,4,50,5,64,2,freedom,1,injured,0.422753
31,3,4,50,5,64,2,freedom,2,act,0.415491
32,3,4,50,5,64,2,freedom,3,nations,0.414656
33,3,4,50,5,64,2,freedom,4,enter,0.412241
34,3,4,50,5,64,2,freedom,5,indeed,0.399403
35,3,4,50,5,64,2,law,1,make,0.497718
36,3,4,50,5,64,2,law,2,people,0.460036
37,3,4,50,5,64,2,law,3,destroy,0.444769
38,3,4,50,5,64,2,law,4,express,0.423579
39,3,4,50,5,64,2,law,5,find,0.421653


- I tried different context window sizes (2,4) and two embedding sizes (50, 100). 
- Qualitatively, the Skip-gram model with a larger context window (4) and 50-dimensional embeddings produced the most coherent neighborhoods for words like freedom, law, and power (e.g., law -> destroy, express; power -> possession, offences). 
- This suggests that a slightly wider context helps the model capture the political and legal themes present in our corpus better than a smaller window size.

## Random Model

In [68]:
def show_neighbors(word, model, vocab, top_k=5, title=""):
    print(f"\n[{title}] neighbors of '{word}':")
    
    if word not in vocab.token2idx:
        print(f"  '{word}' not in vocabulary")
        return

    embeddings = model.emb.weight.detach().cpu()
    neighbors = most_similar_from_embeddings(word, embeddings, vocab, top_k=top_k)

    if not neighbors:
        print("  (no neighbors found)")
        return

    for rank, (nbr, score) in enumerate(neighbors, start=1):
        print(f"  {rank:2d}. {nbr:15s} {score:.4f}")

# Untrained model
random_model = SkipGramModel(len(vocab), EMBEDDING_SIZE).to(device)  

for w in ["freedom", "law", "power"]:
    show_neighbors(w, random_model, vocab, top_k=5, title="random (untrained)")


[random (untrained)] neighbors of 'freedom':
   1. tax             0.2836
   2. conquering      0.2804
   3. neither         0.2647
   4. designed        0.2572
   5. f               0.2557

[random (untrained)] neighbors of 'law':
   1. her             0.3707
   2. damages         0.2936
   3. besides         0.2889
   4. punishment      0.2865
   5. work            0.2798

[random (untrained)] neighbors of 'power':
   1. commonwealths   0.3580
   2. proved          0.3338
   3. prove           0.3295
   4. arbitrary       0.2966
   5. thinks          0.2774


- For the untrained random model, the neighbors of freedom and law are essentially arbitrary content words, with no clear semantic or legal/political theme. A few words like damages, punishment, or commonwealths happen to be loosely related, but overall there is no consistent semantic pattern: the lists mix unrelated verbs, pronouns, and random nouns with similar cosine scores.
- In contrast, trained Skip-gram model produces neighbors that are clearly related to the underlying concepts, showing that the learned embeddings capture meaningful structure that is not present in random embeddings.

### CBOW vs Skip-gram

- CBOW and Skip-gram architectures differ mainly in what they take as input and how they aggregate embeddings.

1. Skip-gram (SkipGramModel):
    - Input: a single target (center) word index per example, shape [batch_size].
    - Forward: emb = self.emb(target_indices) gives [batch_size, embedding_dim], then self.out(emb) produces logits over the whole vocabulary [batch_size, vocab_size] to predict a context word from the center word.

2. CBOW (CBOW):
    - Input: multiple context word indices per example, shape [batch_size, context_len].
    - Forward: self.embeddings(inputs) returns [batch_size, context_len, dims], then mean(dim=1) aggregates all context word embeddings into one vector [batch_size, dims], and self.linear(embeds) outputs [batch_size, vocab_size] to predict the center (target) word.

- In sum, CBOW does “many context words -> average embedding -> predict one target,” while Skip-gram does “one target word -> embedding -> predict a context word.” They have different input shapes and the mean(dim=1) aggregation step only appears in the CBOW forward.